In [2]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Configure Dash to use assets folder one level up
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)
app.title = "🧺 Household Appliance Carousel"

# 📦 Absolute path to the images
APPLIANCE_PATH = os.path.abspath("../assets/object/household_appliance")

# 📦 Load appliance categories
APPLIANCE_ITEMS = []
if os.path.exists(APPLIANCE_PATH):
    APPLIANCE_ITEMS = sorted([
        item for item in os.listdir(APPLIANCE_PATH)
        if os.path.isdir(os.path.join(APPLIANCE_PATH, item))
    ])

# 📦 Build image list
appliance_images = {
    item: sorted([
        f"/assets/object/household_appliance/{item}/{img}"
        for img in os.listdir(os.path.join(APPLIANCE_PATH, item))
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for item in APPLIANCE_ITEMS
}

# 🖼️ Layout
app.layout = html.Div([
    html.H2("🧺 Household Appliance Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select appliance:", style={
            "fontWeight": "normal", "display": "block", "marginBottom": "6px",
            "fontSize": "15px", "fontFamily": "Georgia, serif"
        }),
        dcc.Dropdown(
            id='item-dropdown',
            options=[{"label": item.replace("_", " ").capitalize(), "value": item} for item in APPLIANCE_ITEMS],
            value=APPLIANCE_ITEMS[0] if APPLIANCE_ITEMS else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto"}
        )
    ], style={"textAlign": "center", "marginBottom": "20px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
])

# 🔁 Carousel content
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value')
)
def update_carousel(item):
    images = appliance_images.get(item, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center"})

    filename = os.path.basename(images[0])
    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginRight": "20px"}),
            html.Img(id="carousel-image", src=images[0],
                     style={"height": "450px", "display": "inline-block", "verticalAlign": "middle"}),
            html.Button("➡️", id="next-button", n_clicks=0,
                        style={"fontSize": "24px", "padding": "10px 15px", "marginLeft": "20px"}),
        ], style={"textAlign": "center", "marginBottom": "20px"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),
        html.Div(f"1/{len(images)} - {filename}", id='filename-display',
                 style={"textAlign": "center", "fontSize": "14px", "fontStyle": "italic", "fontFamily": "Georgia, serif"})
    ])

# 🔁 Navigation logic
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    filename = os.path.basename(images[index])
    return images[index], index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Launch app
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8161)
